In [1]:
import pandas as pd
import datetime
from pandas.tseries.offsets import BDay
from dateutil.relativedelta import relativedelta
from collections import defaultdict 
import pprint

from commons import read_xls, get_stock_data, save_csv, get_holding_quantities
from commons import min_consecutive_sum_kadane, get_curr_prices_from_holdings
from consts import HOLDING_FILE

In [2]:
# HOLDINGS_FILE_NAME = '../docs/Holdings_28Dec23.xlsx'

In [57]:
def process_sheets(df, today=None):
    if not today:
        today = datetime.datetime.today()
        
    start = today - BDay(1) # Get prev business date as market didnt start today
    end = today

#     if datetime.time(today.hour, today.minute, today.second) < datetime.time(9, 15, 5):
#         print('Markets didnt start yet, getting data from prev business dates')
#         end = start
#         start -= BDay(2)
    
    try:
        print(f'****** Calling for stock data between {start} and {end} *******')

        curr_prices = get_stock_data(start=start, end=end, print_data=False)
        curr_prices = curr_prices.reset_index().transpose().reset_index().rename(columns={0: 'currPrice'}) #.rename(columns={'index': 'Symbol'})
    except e: 
        print(e)
        print('***** reading current prices from Holdings.txt as yahoo crapped again!!!')
        curr_prices = get_curr_prices_from_holdings()
        
    df = df.merge(curr_prices, left_on=['COMPANY'], right_on=['index'])
    df = df.drop('index', axis=1)
    df = df.sort_values(by='Date', ascending=True)
    df['PnL'] = (df['currPrice'] - df['BuyPrice']) * df['Qty']

#     def flip_sells(t, val):
#         if t == 'sell':
#             val *= -1
            
#         return val

#     df['PnL']      = df.apply(lambda x: flip_sells(x['Trade Type'], x['PnL']), axis=1)
#     df['Quantity'] = df.apply(lambda x: flip_sells(x['Trade Type'], x['Quantity']), axis=1)
    return df

def get_stcl(df, harvest_loss_amount=9999999, today=datetime.datetime.today(), num_years_for_stcg=1):
    '''
        Get the number qty to sell to harvest Short Term Cap Loss of a sum X
    '''
    stcg_cutoff_date = (today - relativedelta(years=num_years_for_stcg)).date()
#     df = df[df['Trade Date'] > str(date)] # Get trades after this date
    df = df.sort_values(by='Date', ascending=False)
    data = {}
    
    for c, q in get_holding_quantities().items():
        print(f'***************** {c}: {q} ************************')
        df_c = df[df['COMPANY'] == c]
        df_dict = df_c.to_dict('list') # https://stackoverflow.com/questions/52547805/how-to-convert-dataframe-to-dictionary-in-pandas-without-index
        ltcg, stcg, ltcg_qty, stcg_qty = 0, 0, 0, 0
        data[c] = defaultdict(int)
        data[c]['total_qty'] = 0
        
        for i, (comp, dt, bp, qty, cp, pnl) in enumerate(zip(df_dict['COMPANY'], df_dict['Date'], df_dict['BuyPrice'], \
                                             df_dict['Qty'], df_dict['currPrice'], df_dict['PnL'])):
            i += 1 
            if data[c]['total_qty'] > q: # Dont process older txns' which were squared off earlier
                print(f"Going back to past quantities... break!: {data[c]['total_qty']} > {q}")
                break 
                
            if dt.date() <= stcg_cutoff_date:
                ### LTCG
#                 print(f'LTCG {i} {dt.date()} >= {stcg_cutoff_date}: {c} Qty={qty} buyPrice={bp} currPrice={cp} pnl={pnl}')
                data[c]['ltcg_qty'] += qty
                data[c]['ltcg'] += pnl
            else:
                ### STCG
#                 print(f'STCG {i} {dt.date()} <  {stcg_cutoff_date}: {c} Qty={qty} buyPrice={bp} currPrice={cp} pnl={pnl}')
                
                # STCLoss only if prev last LTCG txn was negative
                if 'stcg_qty' not in data[c]: # init stcg total qty 
                    data[c]['stcl_qty'] = data[c]['ltcg_qty']
                    data[c]['stcg_qty'] = data[c]['ltcg_qty']

                if pnl < 0: ## only if it is a loss
                    data[c]['stcl_qty'] += qty
                    data[c]['stcl'] += pnl
                else:
                    data[c]['stcg_qty'] += qty
                    data[c]['stcg'] += pnl    
            
                
            data[c]['total_qty'] += qty
            data[c]['total'] += pnl
            
    return data

def get_max_stcl(df, harvest_loss_amount=9999999, today=datetime.datetime.today(), num_years_for_stcg=1):
    '''
        Get the number qty to sell to harvest Short Term Cap Loss of a sum X
    '''
    stcg_cutoff_date = (today - relativedelta(years=num_years_for_stcg)).date()
    data = {}
    
    for c in get_holding_quantities().keys():
        df_c = df[(df['Date'] > str(stcg_cutoff_date)) & (df['COMPANY'] == c)] # Get trades after this date
        df_dict = df_c.to_dict('list')
        data[c] = min_consecutive_sum_kadane(df_dict['PnL'], df_dict['Date'])

    return {k: v for k, v in sorted(data.items(), key=lambda item: item[1])} # sort data by value before returning



In [59]:
df = read_xls(HOLDING_FILE, tab=5)
df['COMPANY'] = df['COMPANY'].apply(lambda x: x.strip())
df = df[df['COMPANY'].isin(get_holding_quantities().keys())]
df = df[['COMPANY', 'Date', 'Qty', 'BuyPrice']]

df = process_sheets(df)
# save_csv(df, 'ltcg_stcg')

print('******* Max STCL - longest STCL loss streak *******')
print(pd.DataFrame(get_max_stcl(df)).T)
final = pd.DataFrame(get_stcl(df)).T.sort_values(by='stcl', ascending=True)
save_csv(final, 'ltcg_stcg')
final

****** Calling for stock data between 2024-05-24 01:56:41.778021 and 2024-05-27 01:56:41.778021 *******
****** Getting stock data between 2024-05-24 01:56:41.778021 and 2024-05-27 01:56:41.778021 *******
[*********************100%%**********************]  23 of 23 completed
****** Got stock data between 2024-05-24 01:56:41.778021 and 2024-05-27 01:56:41.778021 *******
******* Max STCL - longest STCL loss streak *******
                       0          1          2
RAJRATAN   -35349.255737  06Jun2023  13May2024
BAJFINANCE     -14541.75  23Jun2023  23Jan2024
BERGEPAINT -11099.002197  28Sep2023  26Apr2024
HAPPSTMNDS  -9630.153198  23Jun2023  21Feb2024
FINEORG      -8745.14502  22Jun2023  20Jan2024
DEVYANI     -2427.500763  21Aug2023  04Mar2024
PRINCEPIPE      -1558.75  03Oct2023  25Oct2023
IONEXCHANG        -945.0  28Dec2023  28Dec2023
RELAXO       -796.000977  23Jan2024  04Mar2024
KPITTECH     -677.999268  21Feb2024  21Feb2024
SONACOMS      -516.25061  26Apr2024  26Apr2024
DMART        

,total_qty,ltcg_qty,stcl_qty,stcg_qty,stcg,total,stcl,ltcg
RAJRATAN,326.0,91.0,235.0,0.0,NaN,-34137.407959,-35349.255737,1211.847778
BAJFINANCE,82.0,22.0,33.0,27.0,7649.750000,6748.500000,-14541.750000,13640.500000
BERGEPAINT,373.0,161.0,180.0,32.0,15609.599609,-21620.604553,-11099.002197,-26131.201965
FINEORG,64.0,8.0,43.0,13.0,1848.551270,-7047.593750,-9851.945801,955.800781
HAPPSTMNDS,260.0,129.0,131.0,0.0,NaN,-33470.456348,-9630.153198,-23840.303149
DEVYANI,355.0,230.0,125.0,0.0,NaN,-4005.502167,-2427.500763,-1578.001404
PRINCEPIPE,252.0,148.0,45.0,59.0,865.750000,-6031.000000,-1558.750000,-5338.000000
IONEXCHANG,341.0,220.0,30.0,91.0,4048.500000,71750.700000,-945.000000,68647.200000
RELAXO,168.0,128.0,40.0,0.0,NaN,-31541.204102,-796.000977,-30745.203125
KPITTECH,222.0,100.0,15.0,107.0,29020.605225,129151.610840,-677.999268,100809.004883


In [47]:
df1 = read_xls(HOLDING_FILE, tab=5)


In [50]:
set(df1['COMPANY'].tolist())

{'AFFLE',
 'BAJFINANCE',
 'BAJFINANCE ',
 'BEL',
 'BERGEPAINT',
 'BORORENEW',
 'BURGERKING',
 'DEEPAKNTR',
 'DEVYANI',
 'DIVISLAB',
 'DIXON',
 'DMART',
 'FINEORG',
 'GARFIBRES',
 'HAPPSTMNDS',
 'IONEXCHANG',
 'KPITTECH',
 'LAURUSLABS',
 'MOLDTKPAC',
 'NAUKRI',
 'NEOGEN',
 'POLYCAB',
 'PRAJIND',
 'PRINCEPIPE',
 'PRINCEPIPE ',
 'RAJRATAN',
 'RELAXO',
 'SONACOMS',
 'TATACHEM'}

In [53]:
df_obj = df1.select_dtypes('object')
df_obj
df1['COMPANY'] = df1['COMPANY'].apply(lambda x: x.strip())
set(df1['COMPANY'].tolist())

{'AFFLE',
 'BAJFINANCE',
 'BEL',
 'BERGEPAINT',
 'BORORENEW',
 'BURGERKING',
 'DEEPAKNTR',
 'DEVYANI',
 'DIVISLAB',
 'DIXON',
 'DMART',
 'FINEORG',
 'GARFIBRES',
 'HAPPSTMNDS',
 'IONEXCHANG',
 'KPITTECH',
 'LAURUSLABS',
 'MOLDTKPAC',
 'NAUKRI',
 'NEOGEN',
 'POLYCAB',
 'PRAJIND',
 'PRINCEPIPE',
 'RAJRATAN',
 'RELAXO',
 'SONACOMS',
 'TATACHEM'}

In [6]:
df = read_xls('../docs/Holdings_26May24.xlsx', tab=5)
df = df[df['COMPANY'].isin(get_holding_quantities().keys())]
df = df[['COMPANY', 'Date', 'Qty', 'BuyPrice']]
df = process_sheets(df)
# df_dict = df.to_dict('list') # https://stackoverflow.com/questions/52547805/how-to-convert-dataframe-to-dictionary-in-pandas-without-index
# df_dict
df

****** Calling for stock data between 2024-05-24 00:51:26.235800 and 2024-05-27 00:51:26.235800 *******
****** Getting stock data between 2024-05-24 00:51:26.235800 and 2024-05-27 00:51:26.235800 *******
[*********************100%%**********************]  23 of 23 completed
****** Got stock data between 2024-05-24 00:51:26.235800 and 2024-05-27 00:51:26.235800 *******


,COMPANY,Date,Qty,BuyPrice,currPrice,PnL
0,DIXON,2021-04-12,6,3580.0,9304.849609,34349.097656
5,NAUKRI,2021-04-16,4,4601.0,6340.899902,6959.599609
6,NAUKRI,2021-04-23,4,4870.0,6340.899902,5883.599609
21,TATACHEM,2021-05-04,35,729.5,1083.75,12398.75
22,TATACHEM,2021-05-05,15,700.0,1083.75,5756.25
...,...,...,...,...,...,...
266,SONACOMS,2024-04-26,25,651.0,630.349976,-516.25061
296,BERGEPAINT,2024-04-26,36,504.0,487.799988,-583.200439
267,SONACOMS,2024-05-10,35,589.0,630.349976,1447.249146
210,RAJRATAN,2024-05-13,25,590.0,580.849976,-228.75061


In [7]:
# print(s_df)
# s_df.to_dict()
# sum(s_df['Quantity'])
s_df['PnL'] = (s_df['currPrice'] - s_df['Price']) * s_df['Quantity']
print(s_df)
print(sum(s_df[s_df['Trade Type'] == 'buy']['PnL']) - sum(s_df[s_df['Trade Type'] == 'sell']['PnL']))

def f(t, pnl):
    if t == 'sell':
        return -1*pnl
    return pnl

s_df['PnL'] = s_df.apply(lambda x: f(x['Trade Type'], x['PnL']), axis=1)
s_df
# s_df['Trade Type', 'PnL'].apply(lambda x, y: f(x, y))
# s_df[['Trade Type', 'PnL']]

NameError: name 's_df' is not defined

In [ ]:
datetime.datetime.today().minute
# datetime.datetime.today() - BDay(1)
t = datetime.time(9, 30, 59)
t.hour
# datetime.time(datetime.datetime.today())
datetime.time(datetime.datetime.today().hour, datetime.datetime.today().minute) < datetime.time(9, 15, 5)
datetime.datetime.today().second

today = datetime.datetime.today()
today - BDay(2), today - BDay(1) # Get prev business date as market didnt start today

In [ ]:
curr_prices = get_curr_prices_from_holdings()
# print(curr_prices)
df = df.merge(curr_prices, left_on=['COMPANY'], right_on=['index'])
df = df.drop('index', axis=1)
df = df.sort_values(by='Date', ascending=True)
df['PnL'] = (df['currPrice'] - df['BuyPrice']) * df['Qty']
df